In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
class HopfieldNN():
    def train_weights(self, train_data):
        num_data = len(train_data)
        self.num_neuron = train_data[0].shape[0]

        # initialize weights
        W = np.zeros((self.num_neuron, self.num_neuron))
        rho = np.sum([np.sum(t) for t in train_data]) / (num_data*self.num_neuron)

        # Hebb rule
        for i in range(num_data):
            t = train_data[i] - rho
            W += np.outer(t, t)

        # Make diagonal element of W into 0
        diagW = np.diag(np.diag(W))
        W = W - diagW
        W /= num_data

        self.W = W

    def predict(self, data, num_iter=20, threshold=0):
        self.num_iter = num_iter
        self.threshold = threshold

        # Copy to avoid call by reference
        copied_data = np.copy(data)

        # Define predict list
        predicted = []
        for i in range(len(data)):
            predicted.append(self._run(copied_data[i]))
        return predicted

    def _run(self, init_s):
        # Compute initial state energy
        s = init_s

        e = self.energy(s)

        # Iteration
        for i in range(self.num_iter):
            # Update s
            s = np.sign(self.W @ s - self.threshold)
            # Compute new state energy
            e_new = self.energy(s)

            # s is converged
            if e == e_new:
                return s
            # Update energy
            e = e_new
        return s


    def energy(self, s):
        return -0.5 * s @ self.W @ s + np.sum(s * self.threshold)

In [ ]:
import skimage.data
from skimage.color import rgb2gray
from skimage.filters import threshold_mean
from skimage.transform import resize

In [ ]:
# Utils
def get_corrupted_input(input, corruption_level):
    corrupted = np.copy(input)
    inv = np.random.binomial(n=1, p=corruption_level, size=len(input))
    for i, v in enumerate(input):
        if inv[i]:
            corrupted[i] = -1 * v
    return corrupted


def reshape(data):
    dim = int(np.sqrt(len(data)))
    data = np.reshape(data, (dim, dim))
    return data


def plot(data, test, predicted, figsize=(5, 6)):
    data = [reshape(d) for d in data]
    test = [reshape(d) for d in test]
    predicted = [reshape(d) for d in predicted]

    fig, axarr = plt.subplots(len(data), 3, figsize=figsize)
    for i in range(len(data)):
        if i==0:
            axarr[i, 0].set_title('Train')
            axarr[i, 1].set_title('Input')
            axarr[i, 2].set_title('Predict')

        axarr[i, 0].imshow(data[i])
        axarr[i, 0].axis('off')
        axarr[i, 1].imshow(test[i])
        axarr[i, 1].axis('off')
        axarr[i, 2].imshow(predicted[i])
        axarr[i, 2].axis('off')

    plt.tight_layout()
    plt.savefig("result.png")
    plt.show()


def preprocessing(img, w=128, h=128):
    # Resize image
    img = resize(img, (w,h), mode='reflect')

    # Threshold
    thresh = threshold_mean(img)
    binary = img > thresh
    shift = 2 * (binary * 1) - 1

    # Reshape
    flatten = np.reshape(shift, (w*h))
    return flatten


def main():
    # Load data
    page = skimage.data.page()
    rocket = rgb2gray(skimage.data.rocket())
    coins = skimage.data.coins()
    coffee = rgb2gray(skimage.data.coffee())

    # Marge data
    data = [page, rocket, coins, coffee]

    # Preprocessing
    data = [preprocessing(d) for d in data]

    # Create Hopfield Network Model
    model = HopfieldNN()
    model.train_weights(data)

    # Generate testset
    test = [get_corrupted_input(d, 0.3) for d in data]

    predicted = model.predict(test, threshold=0)
    plot(data, test, predicted)

In [ ]:
main()